![QuantConnect Logo](https://cdn.quantconnect.com/web/i/icon.png)
<hr>

# <center> THIS RESEARCH NOTEBOOK IS FOR DOCUMENTATION PURPOSE ONLY <center>

In [1]:
%matplotlib inline
from clr import AddReference
AddReference("System")
AddReference("QuantConnect.Common")
AddReference("QuantConnect.Jupyter")
AddReference("QuantConnect.Indicators")
from System import *
from QuantConnect import *
from QuantConnect.Data.Market import TradeBar, QuoteBar
from QuantConnect.Jupyter import *
from QuantConnect.Indicators import *
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
# create an instance
qb = QuantBook()
plt.style.use('seaborn-whitegrid')

## Alpha Logic - Long-Short Moving Average Crossover

The purpose of this research notebook is to illustrate and test the logic of the alpha model that will be used in the algorithm. Sometimes, an algorithm can feel like a 'black box' so we are basically recreating the way the backtesting engine will run in order to understand how the trading signals are generated. This is a good way to test the logic of an alpha as the code can then be migrated to the Alpha module for backtesting.

In this case, we are building a simple Moving Average Crossover system that will generate a Buy Signal to go long when the Short SMA crosses above the Long SMA, and a Sell Signal to go Short when it crosses below.

The class SymbolData below can be copied into the algorithm's Alpha model for backtesting without any changes required.

In [2]:
# THIS CLASS CAN BE COPIED INTO AN ALPHA MODEL FOR BACKTESTING WITHOUT CHANGES
    # The class is initialized with a symbol object, the period for the short SMA and the period for the long SMA
    # Then, it contains the function UpdateIndicators that simply takes a history dataframe
    # and makes all the needed calculations to create the trading signals

class SymbolData:
    
    '''
    make all the calculations needed for each symbol including
    all the indicators and whether the ticker meets the criteria
    '''
    
    def __init__(self, symbol, shortPeriodSMA, longPeriodSMA):
        self.Symbol = symbol
        self.shortPeriod = shortPeriodSMA
        self.longPeriod = longPeriodSMA
        self.closePrices = RollingWindow[float](longPeriodSMA + 1)
    
    # method to update the rolling window
    def UpdateIndicators(self, history):
        if str(self.Symbol) in history.index:
            for index, row in history.loc[str(self.Symbol)].iterrows():
                if 'close' in row:
                    self.closePrices.Add(row['close'])
                else:
                    raise Exception('missing some close prices for: ' + str(self.Symbol.Value))
        else:
            raise Exception('symbol not in history index: ' + str(self.Symbol.Value))
    
    # convert the rolling window to list for easier manipulation
    @property
    def listClosePrices(self):
        if self.closePrices.IsReady:
            return [float(x) for x in self.closePrices]
        else:
            return [0]
    
    # update short and long current SMA
    @property
    def currentShortSMA(self):
        return np.mean(self.listClosePrices[:self.shortPeriod])
    @property
    def currentLongSMA(self):
        return np.mean(self.listClosePrices[:self.longPeriod])
    
    # update short and long before SMA (the SMA from the previous trading bar)
    @property
    def beforeShortSMA(self):
        return np.mean(self.listClosePrices[1:][:self.shortPeriod])
    @property
    def beforeLongSMA(self):
        return np.mean(self.listClosePrices[1:][:self.longPeriod])
    
    # update boolean for cross above/below of moving averages
    @property
    def crossAbove(self):
        return (self.currentShortSMA > self.currentLongSMA) and (self.beforeShortSMA < self.beforeLongSMA)
    @property
    def crossBelow(self):
        return (self.currentShortSMA < self.currentLongSMA) and (self.beforeShortSMA > self.beforeLongSMA)

In [2]:
# THIS IS SIMILAR TO THE PROCESS OF INITIALIZING THE DATES FOR THE BACKTEST AND THE SECURITIES TO TRADE

# inputs for model --------------------

startDate = datetime(2005, 1, 1)
endDate = datetime(2019, 11, 1)

tickers = ['SPY', 'FB']
shortPeriodSMA = 50
longPeriodSMA = 200

# -------------------------------------

symbolsDict = {}

# adding data to the tickers
for ticker in tickers:
    symbolsDict[ticker] = qb.AddEquity(ticker)

# we use this to loop through the trading days as the backtesting engine would do
market = qb.AddEquity('SPY').Symbol
marketHistory = qb.History(market, startDate, endDate, Resolution.Daily)
marketIndex = marketHistory.loc[market].index

In [ ]:
# THIS IS SIMILAR TO THE WAY THE ALPHA MODEL WORKS, RUNNING EVERY TIME THERE IS NEW DATA,
# UPDATING ALL THE INDICATORS AND GENERATING THE SIGNALS FOR TRADING

calculations = {}
dataCharts = {key:{'symbolsIndex': [], 'shortSMA':[], 'longSMA':[], 'buySignal':[], 'sellSignal':[]}
              for key in qb.Securities.Keys}

# iterate over each simulated day
for date in marketIndex[longPeriodSMA + 1:]:
    datePosition = marketIndex.get_loc(date)
    
    ### Event 1) Firstly, we are going to add the necessary data to refresh indicators and calculate the crossover ---------
    
    # get the symbols for which we have already calculate indicators to simply add last data point to update them
    # we separate this from new symbols to avoid calling full history for all securities every time
    currentSymbols = [symbolsDict[x].Symbol for x in tickers if symbolsDict[x].Symbol in calculations.keys()]
    
    if len(currentSymbols) > 0:
        # add the last data point if the symbol is already in calculations
        historyCurrentSymbols = qb.History(currentSymbols, marketIndex[datePosition - 1], date, Resolution.Daily)

    # get the new symbols for which we need to warm up indicators from scratch
    newSymbols = [symbolsDict[x].Symbol for x in tickers if symbolsDict[x].Symbol not in calculations.keys()]
    if len(newSymbols) > 0:
        historyNewSymbols = qb.History(newSymbols, marketIndex[datePosition - longPeriodSMA - 1], date, Resolution.Daily)

    # now loop through securities to create/update indicators
    for ticker in tickers:
        symbol = symbolsDict[ticker].Symbol # get the symbol
        
        if symbol in newSymbols:
            calculations[symbol] = SymbolData(symbol, shortPeriodSMA, longPeriodSMA)
            history = historyNewSymbols
        else:
            history = historyCurrentSymbols

        try:
            calculations[symbol].UpdateIndicators(history)
        except Exception:
            calculations.pop(symbol, None)
            continue
            
    ### ------------------------------------------------------------------------------------------------------------------
    
    ### Event 2) Secondly, we are going to check for signals and generate buy/sell signals when needed -------------------
    
    # loop through calculations
    for symbol, symbolData in calculations.items():
        if symbolData.crossAbove:
            # here we would generate a signal to go long
            dataCharts[symbol]['buySignal'].append(symbolData.currentShortSMA)
            dataCharts[symbol]['sellSignal'].append(np.nan)

        elif symbolData.crossBelow:
            # here we would generate a signal to close our long position
            dataCharts[symbol]['sellSignal'].append(symbolData.currentShortSMA)
            dataCharts[symbol]['buySignal'].append(np.nan)
        
        elif symbolData.closePrices.IsReady:
            # update data for plots
            dataCharts[symbol]['buySignal'].append(np.nan)
            dataCharts[symbol]['sellSignal'].append(np.nan)

        if symbolData.closePrices.IsReady:
            # update data for plots
            dataCharts[symbol]['shortSMA'].append(symbolData.currentShortSMA)
            dataCharts[symbol]['longSMA'].append(symbolData.currentLongSMA)
            dataCharts[symbol]['symbolsIndex'].append(date)
    
    ### -------------------------------------------------------------------------------------------------------------------

In [ ]:
# PLOTTING THE LOGIC FOR EACH SYMBOL

for symbol, symbolData in calculations.items():
    df = pd.DataFrame({'shortSMA': dataCharts[symbol]['shortSMA'], 'longSMA': dataCharts[symbol]['longSMA'],
                       'buySignal': dataCharts[symbol]['buySignal'], 'sellSignal': dataCharts[symbol]['sellSignal']
                      })
    df = df.set_index(pd.DatetimeIndex(dataCharts[symbol]['symbolsIndex']))

    plt.figure(figsize = (20, 10))
    plt.title(str(symbol) + ' - Moving Average Crossover')
    plt.plot(df['shortSMA'], label = 'Short SMA')
    plt.plot(df['longSMA'], label = 'Long SMA')
    plt.scatter(df.index, df['buySignal'], color = 'green', marker = '^', label = 'Buy Signal')
    plt.scatter(df.index, df['sellSignal'], color = 'red', marker = 'v', label = 'Sell Signal')
    plt.legend()